# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [59]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import gc
import cv2

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from keras.layers import Conv2D, LSTM, Dense, Flatten, MaxPooling2D, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import fbeta_score
from sklearn.utils import shuffle

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
from PIL import Image
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History
#os.listdir('/kaggle/input/')

In [50]:
#Read the data
train = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sample = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')

In [36]:
train.shape

(40479, 2)

In [ ]:
train.head()

In [ ]:
train['tags'].nunique()

In [46]:
sample.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [38]:
train['tags'].nunique()

449

In [51]:
labels = sorted({ee for e in train.tags.unique() for ee in e.split(' ')})
labels

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

for label in labels:
    train[label] = train.tags.str.contains(label)
train.head(10)

In [52]:
label_map = {'agriculture': 0, 'artisinal_mine': 1, 'bare_ground': 2,
             'blooming': 3, 'blow_down': 4, 'clear': 5, 'cloudy': 6,
             'conventional_mine': 7, 'cultivation': 8, 'habitation': 9,
             'haze': 10, 'partly_cloudy': 11, 'primary': 12, 'road': 13,
             'selective_logging': 14, 'slash_burn': 15, 'water': 16}

In [53]:
#SHuffle data
X = []
Y = []
train = shuffle(train,random_state=42)

In [54]:
for image_name, tags in tqdm(train.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)

100%|██████████| 40479/40479 [04:57<00:00, 136.23it/s]


In [55]:
X = np.array(X, np.float16)/255.0

In [57]:
# Horizontal split
X = np.array(X)
Y = np.array(Y)
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 42)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [58]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [64]:
tf.random.set_seed(42)
model = Sequential()

model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(17, activation='relu'))
model.compile(optimizer=Adam(), loss=tf.keras.losses.categorical_crossentropy,metrics=[fbeta])
model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=20, batch_size=1000)

Epoch 1/20
33/33 [==============================] - 29s 870ms/step - loss: 11.3656 - fbeta: 0.6281 - val_loss: 11.2798 - val_fbeta: 0.6530
Epoch 2/20
33/33 [==============================] - 29s 869ms/step - loss: 11.0759 - fbeta: 0.6561 - val_loss: 11.2442 - val_fbeta: 0.6773
Epoch 3/20
33/33 [==============================] - 29s 873ms/step - loss: 10.9882 - fbeta: 0.6912 - val_loss: 11.1089 - val_fbeta: 0.6972
Epoch 4/20
33/33 [==============================] - 29s 870ms/step - loss: 10.8986 - fbeta: 0.7063 - val_loss: 11.0544 - val_fbeta: 0.7146
Epoch 5/20
33/33 [==============================] - 28s 863ms/step - loss: 10.9421 - fbeta: 0.7022 - val_loss: 11.6404 - val_fbeta: 0.6549
Epoch 6/20
33/33 [==============================] - 28s 841ms/step - loss: 11.0715 - fbeta: 0.6755 - val_loss: 11.1015 - val_fbeta: 0.6878
Epoch 7/20
33/33 [==============================] - 29s 866ms/step - loss: 10.8644 - fbeta: 0.7037 - val_loss: 11.0506 - val_fbeta: 0.7182
Epoch 8/20
33/33 [=========

#Encode categorical variables
encoder=LabelEncoder()
encoder.fit_transform(train['tags'])

In [70]:
gc.collect()

1376

In [69]:
#with tpu_strategy.scope():

test_loss, test_accuracy = model.evaluate(X_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

253/253 [==============================] - 3s 13ms/step - loss: 11.2696 - fbeta: 0.6520
Test loss: 11.269633293151855
Test accuracy: 0.6520043611526489


In [72]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sample[0 : 40669]
files = sample[40669 : ]#with tpu_strategy.scope():  

test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [02:38<00:00, 129.30it/s]


In [73]:
gc.collect()

31

In [77]:
#with tpu_strategy.scope():

y_pred = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
y_pred.append(predictions)

957/957 - 17s


In [76]:
#converting my encoded labels back to it original form

sub = np.array(y_pred[0])
for i in range (1, len(y_pred)):
    sub += np.array(y_pred[i])
sub = pd.DataFrame(sub, columns = label_map)

In [78]:
sub.head()

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,60.900421,0.0,0.0,0.0,0.0,141.069489,10.896940,0.0,0.0,0.0,12.898143,35.810780,186.459747,0.0,0.0,0.0,36.964733
1,40.786316,0.0,0.0,0.0,0.0,94.779999,7.290158,0.0,0.0,0.0,8.648997,24.005835,125.267258,0.0,0.0,0.0,24.759209
2,98.947502,0.0,0.0,0.0,0.0,228.841156,17.556166,0.0,0.0,0.0,20.430218,58.918247,303.054077,0.0,0.0,0.0,60.250389
3,49.825771,0.0,0.0,0.0,0.0,115.308136,8.864174,0.0,0.0,0.0,10.545156,29.428177,152.572937,0.0,0.0,0.0,30.244286
4,116.022804,0.0,0.0,0.0,0.0,265.047516,20.223793,0.0,0.0,0.0,22.837709,69.635323,351.269135,0.0,0.0,0.0,70.858345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,107.770729,0.0,0.0,0.0,0.0,249.273026,19.089981,0.0,0.0,0.0,22.712629,63.233429,329.454529,0.0,0.0,0.0,65.268181
61187,58.153255,0.0,0.0,0.0,0.0,133.663971,9.986351,0.0,0.0,0.0,12.386266,34.445625,177.520416,0.0,0.0,0.0,35.177059
61188,42.747677,0.0,0.0,0.0,0.0,99.137405,7.616094,0.0,0.0,0.0,9.020967,25.200108,131.090088,0.0,0.0,0.0,25.953470
61189,126.815720,0.0,0.0,0.0,0.0,293.571838,22.817736,0.0,0.0,0.0,26.904673,74.684601,388.159973,0.0,0.0,0.0,77.004822


In [79]:
preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sample['tags'] = preds
sample.to_csv('submission.csv', index=False)

100%|██████████| 61191/61191 [01:58<00:00, 516.55it/s]


Adapted from https://www.kaggle.com/code/greeshuuu/analyzing-amazon-using-cnn#Image-Classification-Using-The-Neural-Network